# Data Transformation

This notebook focuses on transforming cleaned data into analysis-ready features.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
patients_cleaned = pd.read_csv("data/patients_cleaned.csv")

In [ ]:
patients_cleaned.head()

In [ ]:
patients_cleaned.info()

## Checking categorical variables

- Categorical variables often contain multiple spellings or encodings for the same concept. We need to standardize these.
- Here we focus on the `gender` column:
    - First, we inspect unique values and their counts.
    - Then we replace inconsistent entries with a standard value.
- Casting to `category` dtype can reduce memory and make intent explicit.

In [ ]:
# Show number of unique categories and their counts
patients_cleaned['gender'].value_counts(dropna=False)

In [ ]:
# Correct inconsistent entries - f -> female, m -> male
patients_cleaned['gender'] = patients_cleaned['gender'].replace('f', 'female')
patients_cleaned['gender'] = patients_cleaned['gender'].replace('m', 'male')

In [ ]:
patients_cleaned['gender'].value_counts(dropna=False)

In [ ]:
# visualize gender distribution

### Check admission_unit for unique values

In [ ]:
# Check unique values in admission_unit
patients_cleaned['admission_unit'].value_counts(dropna=False)

### Rename inconsistent entries

In [ ]:
# Standardize admission_unit entries
patients_cleaned['admission_unit'] = patients_cleaned['admission_unit'].replace({'Emergency Room': 'ER', 'Intensive Care U.': 'ICU', 'Cardiol.': 'Cardiology'})

In [ ]:
# visualize admission_unit distribution

plt.figure(figsize=(8,4))
sns.countplot(data=patients_cleaned, x='admission_unit', order=patients_cleaned['admission_unit'].value_counts().index)
plt.title('Admission Unit Distribution')
plt.ylabel('Number of Patients')
plt.xlabel('Admission Unit')
plt.xticks(rotation=45)
plt.show()

## Encoding categorical variables for modeling

- Many models require numeric inputs. `pd.get_dummies()` creates one-hot encoded columns for categorical variables.
- For high-cardinality categorical features you may want alternative encoding strategies (target encoding, embedding, hashing).

### One-hot encoding for 'gender'

In [ ]:
# One-hot encode gender
patients_cleaned = pd.get_dummies(patients_cleaned, columns=['gender'], drop_first=True)

### Label Encoding for 'admission_unit'

- For admission_unit with multiple categories, we use label encoding to convert categories to integer codes.
- This is simple but imposes an ordinal relationship. For non-ordinal categories, one-hot encoding is often preferred.
- Here we demonstrate label encoding for variety.
- First convert to 'category' dtype, then use `.cat.codes` to get integer codes.
- Note: In practice, use sklearn's LabelEncoder or OrdinalEncoder for more control.

In [ ]:
# Convert admission_unit to 'category' dtype
patients_cleaned['admission_unit_encoded'] = patients_cleaned['admission_unit'].astype('category')

# Label encode admission_unit
patients_cleaned['admission_unit_encoded'] = patients_cleaned['admission_unit_encoded'].cat.codes

In [ ]:
patients_cleaned.head()

## Feature engineering - creating new features from existing ones

- BMI (Body Mass Index) is a common clinical feature derived from weight and height.
- BMI = weight (kg) / (height (m))2
- In our dataset height is in cm, so it needs to be converted before calculation.


In [ ]:
# Compute BMI (height in cm -> convert to meters)
patients_cleaned['BMI'] = patients_cleaned['weight'] / (patients_cleaned['height']/100)**2

In [ ]:
# Inspect BMI distribution and missingness
patients_cleaned['BMI'].describe()

### Save transformed data

In [ ]:
# Save the transformed dataset
patients_cleaned.to_csv("data/patients_transformed.csv", index=False)

## Scaling and standardization

- Many ML algorithms assume features are on similar scales
- `MinMaxScaler` rescales to [0,1]
- `StandardScaler` centers to mean=0 and std=1

We demonstrate both for different use cases.

### Method 1: Min-Max Scaling

In [ ]:
# Demonstrate MinMax scaling for 'age' and Standard scaling for blood pressure
minmax = MinMaxScaler()
patients_cleaned['age_minmax'] = minmax.fit_transform(patients_cleaned[['age']])

### Method 2: Standardization

In [ ]:
std = StandardScaler()
patients_cleaned['bp_std'] = std.fit_transform(patients_cleaned[['blood_pressure']])

In [ ]:
patients_cleaned.head()

### Further preprocessing for modeling

- scaling all numeric features
- dropping any remaining irrelevant columns
- handling any remaining missing values
- drop identifier `patient_id`

## Exercise — Data Transformation

1. Standardize the `blood_pressure` column (use StandardScaler - is already imported), store as `bp_standardized`.
2. Create a new feature `length_of_stay` as the difference in days between `discharge_date` and `admission_date`. (Hint: columns need to be datetime dtype)
3. Plot a histogram of the computed BMI.


In [ ]:
# 1. Standardize blood_pressure

In [ ]:
# 2. Create length_of_stay feature

In [ ]:
# 3. Plot histogram of BMI